## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,2022-09-03 01:56:55,-21.3585,55.7679,63.99,83,40,9.42,light rain
1,1,Mtimbira,TZ,2022-09-03 01:56:55,-8.7833,36.3500,62.92,61,4,1.88,clear sky
2,2,Hobart,AU,2022-09-03 01:56:56,-42.8794,147.3294,57.06,59,75,4.61,broken clouds
3,3,Digkilaan,PH,2022-09-03 01:56:56,8.2511,124.3172,82.65,65,100,3.85,overcast clouds
4,4,Jamestown,US,2022-09-03 01:56:56,42.0970,-79.2353,72.72,59,100,4.00,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Digkilaan,PH,2022-09-03 01:56:56,8.2511,124.3172,82.65,65,100,3.85,overcast clouds
7,7,Vaini,TO,2022-09-03 01:56:57,-21.2000,-175.2000,75.36,78,20,12.66,few clouds
10,10,Itoman,JP,2022-09-03 01:56:58,26.1247,127.6694,81.37,92,75,27.63,light intensity shower rain
18,18,Kapaa,US,2022-09-03 01:57:01,22.0752,-159.3190,83.48,67,7,7.00,clear sky
19,19,Mareeba,AU,2022-09-03 01:57:01,-17.0000,145.4333,78.48,52,30,5.32,scattered clouds
27,27,Tura,IN,2022-09-03 01:52:06,25.5198,90.2201,76.57,90,70,7.23,light rain
28,28,Pemberton,CA,2022-09-03 01:57:03,50.3164,-122.8027,83.05,30,0,5.10,clear sky
31,31,Hilo,US,2022-09-03 01:57:04,19.7297,-155.0900,87.28,84,3,3.00,clear sky
41,41,Mujiayingzi,CN,2022-09-03 01:58:07,42.1167,118.7833,76.33,25,92,2.86,overcast clouds
42,42,Salalah,OM,2022-09-03 01:58:07,17.0151,54.0924,76.41,84,88,6.33,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().count() - preferred_cities_df.count()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_df['City'].replace('', np.nan, inplace=True)

clean_df = preferred_cities_df.dropna()

clean_df
clean_df = preferred_cities_df

C:\Users\Andre\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Digkilaan,PH,82.65,overcast clouds,8.2511,124.3172,
7,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
10,Itoman,JP,81.37,light intensity shower rain,26.1247,127.6694,
18,Kapaa,US,83.48,clear sky,22.0752,-159.3190,
19,Mareeba,AU,78.48,scattered clouds,-17.0000,145.4333,
27,Tura,IN,76.57,light rain,25.5198,90.2201,
28,Pemberton,CA,83.05,clear sky,50.3164,-122.8027,
31,Hilo,US,87.28,clear sky,19.7297,-155.0900,
41,Mujiayingzi,CN,76.33,overcast clouds,42.1167,118.7833,
42,Salalah,OM,76.41,overcast clouds,17.0151,54.0924,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFram
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    hotels
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
len(hotels["results"])

20

In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Digkilaan,PH,82.65,overcast clouds,8.2511,124.3172,VILLAGRACIA WHITEWATER RAFT RESORT
7,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Itoman,JP,81.37,light intensity shower rain,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
18,Kapaa,US,83.48,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Mareeba,AU,78.48,scattered clouds,-17.0000,145.4333,Trinity Plains Tourist Park
...,...,...,...,...,...,...,...
673,Dicabisagan,PH,82.15,overcast clouds,17.0818,122.4157,Nancy Montanes Bernardo
677,Umm Lajj,SA,83.66,overcast clouds,25.0213,37.2685,Moon light Furnished Units
686,Hami,CN,78.19,clear sky,42.8000,93.4500,Laoyutang Resort
689,Muskegon,US,75.06,broken clouds,43.3000,-86.1334,Timberline Estates


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
curr_desc = clean_hotel_df["Hotel Name"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))